In [1]:
import torch
import torch.nn as nn
import numpy as np
import sys; sys.path.append('../src/')
import os
from model.cmpnts import MLP
from scipy.stats import ortho_group
import matplotlib.pyplot as plt
from math import sqrt

In [2]:
data_dir = '../data/toy'

In [3]:
def nonlinearity(dataset, epsilon=1.): # epsilon can be set to the groundtruth LD manifold's average std.
    X = dataset[:]
    _, s, _ = torch.svd(X - X.mean(0), compute_uv=False)
    return s.max() / s.min(), s.min() / epsilon, epsilon, s

In [4]:
def save_dataset(X, Y, i):
    in_dim = X.shape[1]
    out_dim = Y.shape[1]
    path = os.path.join(data_dir, '{}-manifold'.format(in_dim), '{}-ambient'.format(out_dim))
    os.makedirs(path, exist_ok=True)
    
    data_path = os.path.join(path, '{}-{}_{}.npy'.format(in_dim, out_dim, i))
    report_path = os.path.join(path, '{}-{}_{}.txt'.format(in_dim, out_dim, i))
    np.save(data_path, Y.detach().numpy())
    
    _, s_x, _ = torch.svd(X - X.mean(0), compute_uv=False)
    dis, ratio, eps, s = nonlinearity(Y, epsilon=s_x.mean())
    with open(report_path, 'w') as report:
        report.write('s.max / s.min = {}\n'.format(dis))
        report.write('s.min / x.s.mean = {}\n'.format(ratio))
        report.write('x.s.mean = {}\n'.format(eps / sqrt(len(X))))
        report.write('singular values = {}'.format(s.sort()[0].flip(0) / sqrt(len(X))))

# Nonlinear Dataset

In [5]:
from dataset.toy import IsometricEmbedding
from model.flow import _RealNVP

In [12]:
def build_flow(out_dim, n=10, w=256):
    nets = lambda: nn.Sequential(
        nn.Linear(out_dim, w), nn.ReLU(), 
        nn.Linear(w, w), nn.ReLU(),
        nn.Linear(w, w), nn.ReLU(),
        nn.Linear(w, out_dim), nn.Tanh())
    nett = lambda: nn.Sequential(
        nn.Linear(out_dim, w), nn.ReLU(), 
        nn.Linear(w, w), nn.ReLU(),
        nn.Linear(w, w), nn.ReLU(),
        nn.Linear(w, out_dim))
    masks = torch.as_tensor([
        [0]*(out_dim//2) + [1]*(out_dim//2), 
        [1]*(out_dim//2) + [0]*(out_dim//2)] * n, dtype=torch.float)
    flow = _RealNVP(nets, nett, masks)
    return flow

In [19]:
from tqdm import tqdm

in_dims = [2, 4, 8, 16, 32, 64] # [1, 2, 4, 8, 16, 32, 64]
out_dims = np.asarray([2, 4, 16, 64, 256], dtype=int)
sizes = [300, 1000, 3000, 10000, 30000, 100000]

for in_dim, size in zip(in_dims, sizes):
    for out_dim in out_dims * in_dim:
        for i in range(5):
            print('{} -> {} ({})'.format(in_dim, out_dim, i))

            X = torch.randn(size, in_dim)
            Y = IsometricEmbedding(X, out_dim=out_dim)

            flow = build_flow(out_dim, 10, 256)
            Y.embed(flow = flow,
                    optimizer = torch.optim.Adam(flow.parameters(), lr=1e-4),
                    epochs = 100 * int(sqrt(out_dim / in_dim)),
                    batch_size = 100,
                    lam = 100.)

            save_dataset(X, Y[:], i)

4 -> 8 (0)


Embedding: 100%|██████████████████████████████| 100/100 [00:16<00:00,  6.12it/s, isometricity=0.104, linearity=0.265]


4 -> 8 (1)


Embedding: 100%|██████████████████████████████| 100/100 [00:15<00:00,  6.35it/s, isometricity=0.0913, linearity=0.242]


4 -> 8 (2)


...

32 -> 8192 (3)


Embedding: 100%|██████████████████████████████| 1600/1600 [50:19<00:00,  1.89s/it, isometricity=0.0378, linearity=0.231]


32 -> 8192 (4)


Embedding:  54%|████████████████▎             | 868/1600 [24:37<20:46,  1.70s/it, isometricity=0.0394, linearity=0.264]


KeyboardInterrupt: 